# 案例: 水力发电仿真

本案例演示了一个基础的水力发电场景。系统拓扑如下：

`上游湖泊 (Lake) -> 水轮机 (WaterTurbine) -> 下游渠道 (Canal)`

我们将使用`SimulationHarness`来搭建和运行这个仿真，所有组件和设置均由一个JSON配置文件定义。

### 1. 导入库并加载配置

In [ ]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

from swp.simulation_identification.physical_objects.lake import Lake
from swp.simulation_identification.physical_objects.water_turbine import WaterTurbine
from swp.simulation_identification.physical_objects.canal import Canal
from swp.core_engine.testing.simulation_harness import SimulationHarness

CONFIG_PATH = 'examples/hydropower_simulation.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义并运行仿真

In [ ]:
def run_hydropower_simulation(cfg):
    """根据配置搭建并运行水电仿真。"""
    print(f"--- 开始从配置文件加载仿真: {CONFIG_PATH} ---")
    
    COMPONENT_CLASS_MAP = {
        "Lake": Lake,
        "WaterTurbine": WaterTurbine,
        "Canal": Canal,
    }

    # 1. 设置仿真平台
    harness = SimulationHarness(config=cfg['simulation_settings'])

    # 2. 动态实例化并添加组件
    print("\n--- 正在实例化组件 ---")
    for comp_conf in cfg['components']:
        comp_id, comp_type = comp_conf['id'], comp_conf['type']
        CompClass = COMPONENT_CLASS_MAP[comp_type]
        instance = CompClass(name=comp_id, initial_state=comp_conf.get('initial_state', {}), parameters=comp_conf.get('params', {}))
        
        # 假设水轮机总是以最大容量运行以简化示例
        if comp_type == "WaterTurbine":
            instance.target_outflow = instance.max_flow_rate
            
        harness.add_component(instance)
        print(f"  - 已创建 {comp_type}: {comp_id}")

    # 3. 添加连接
    print("\n--- 正在连接组件 ---")
    for conn in cfg['connections']:
        harness.add_connection(conn['from'], conn['to'])
    print("  - 连接已建立。")
    
    # 4. 构建并运行仿真
    harness.build()
    harness.run_simulation()

    # 5. 保存结果到JSON文件
    output_path = os.path.join(os.path.dirname(CONFIG_PATH), "hydropower_output.json")
    with open(output_path, 'w') as f:
        json.dump(harness.history, f, indent=4)
    print(f"\n--- 结果已保存至 {output_path} ---")
    return harness.history

history = run_hydropower_simulation(config)

### 3. 可视化结果

In [ ]:
def plot_hydropower_results(simulation_history):
    if not simulation_history:
        print("没有历史数据可供绘制。")
        return

    # 将历史记录转换为Pandas DataFrame
    df = pd.DataFrame()
    for step in simulation_history:
        time = step['time']
        row = {'time': time}
        for comp, states in step.items():
            if comp == 'time': continue
            for state, value in states.items():
                row[f"{comp}.{state}"] = value
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df = df.set_index('time')

    fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    fig.suptitle('水电仿真结果', fontsize=16)

    # 绘制湖泊水位
    axes[0].plot(df.index, df['lake.water_level'], label='湖泊水位 (m)')
    axes[0].set_title('上游湖泊水位变化')
    axes[0].set_ylabel('水位 (m)')
    axes[0].grid(True)
    axes[0].legend()

    # 绘制水轮机流量
    axes[1].plot(df.index, df['turbine.outflow'], label='水轮机流量 (m³/s)', color='green')
    axes[1].set_title('水轮机发电流量')
    axes[1].set_ylabel('流量 (m³/s)')
    axes[1].set_xlabel('时间 (s)')
    axes[1].grid(True)
    axes[1].legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

plot_hydropower_results(history)